In [ ]:
import os
import json
import numpy as np
from tqdm import tqdm
from transformers import BertTokenizer, BertModel


In [ ]:
BASE_PATH = "/content/drive/MyDrive/NLP/CA5"
RAW_DATA_PATH = os.path.join(BASE_PATH, "raw_data")
TOKENIZED_DATA_PATH = os.path.join(BASE_PATH, "tokenized_data")

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained("bert-base-multilingual-cased")
text = "This is a test"
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

In [1]:
# Read every file inside raw_data directory seperately and split using line breaks and save it to corresponding variable

with open(os.path.join(RAW_DATA_PATH, "train.en"), 'r') as f:
    train_en = f.read().splitlines()

with open(os.path.join(RAW_DATA_PATH, "train.fa"), 'r') as f:
    train_fa = f.read().splitlines()

with open(os.path.join(RAW_DATA_PATH, "valid.en"), 'r') as f:
    valid_en = f.read().splitlines()

with open(os.path.join(RAW_DATA_PATH, "valid.fa"), 'r') as f:
    valid_fa = f.read().splitlines()

with open(os.path.join(RAW_DATA_PATH, "test.en"), 'r') as f:
    test_en = f.read().splitlines()

with open(os.path.join(RAW_DATA_PATH, "test.fa"), 'r') as f:
    test_fa = f.read().splitlines()



# Now for each file iterate over the list and perform tokenizer function on each line and save it to corresponding variable and add description to each running tqdm loop

train_en_tokenized = [tokenizer(line, return_tensors='pt').input_ids for line in tqdm(train_en, desc="train_en")]
train_fa_tokenized = [tokenizer(line, return_tensors='pt').input_ids for line in tqdm(train_fa, desc="train_fa")]
valid_en_tokenized = [tokenizer(line, return_tensors='pt').input_ids for line in tqdm(valid_en, desc="valid_en")]
valid_fa_tokenized = [tokenizer(line, return_tensors='pt').input_ids for line in tqdm(valid_fa, desc="valid_fa")]
test_en_tokenized = [tokenizer(line, return_tensors='pt').input_ids for line in tqdm(test_en, desc="test_en")]
test_fa_tokenized = [tokenizer(line, return_tensors='pt').input_ids for line in tqdm(test_fa, desc="test_fa")]



# Now write the tokenized data to files in mBert_tokenized_data directory, each item will be seperated by line break

with open(os.path.join(TOKENIZED_DATA_PATH, "train.en"), 'w') as f:
    f.writelines(train_en_tokenized)

with open(os.path.join(TOKENIZED_DATA_PATH, "train.fa"), 'w') as f:
    f.writelines(train_fa_tokenized)

with open(os.path.join(TOKENIZED_DATA_PATH, "valid.en"), 'w') as f:
    f.writelines(valid_en_tokenized)

with open(os.path.join(TOKENIZED_DATA_PATH, "valid.fa"), 'w') as f:
    f.writelines(valid_fa_tokenized)

with open(os.path.join(TOKENIZED_DATA_PATH, "test.en"), 'w') as f:
    f.writelines(test_en_tokenized)

with open(os.path.join(TOKENIZED_DATA_PATH, "test.fa"), 'w') as f:
    f.writelines(test_fa_tokenized)
